### 人脸面部特征识别

In [1]:
import pandas as pd
import numpy as np
import zipfile
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import transforms


with zipfile.ZipFile('test.zip', 'r') as zip_ref:
    zip_ref.extractall('test_data')
    
with zipfile.ZipFile('training.zip', 'r') as zip_ref:
    zip_ref.extractall('training_data')
    
train = pd.read_csv('training_data/training.csv')
test = pd.read_csv('test_data/test.csv')
label = pd.read_csv('IdLookupTable.csv')

In [2]:
# 建立网络结构
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=(3, 3), stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(128), 
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.BatchNorm2d(128), 
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=(3, 3), stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(2, 2)),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=(3, 3), stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.flatten = nn.Flatten()
        self.dense1 = nn.Sequential(
            nn.Linear(256*6*6, 512),
            nn.ReLU(),
            nn.Dropout(p=0.1)
        )
        self.dense2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1)
        )
        self.dense3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.dense4 = nn.Linear(128, 30)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return x

In [3]:
train.head()

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...
3,65.225739,37.261774,32.023096,37.261774,60.003339,39.127179,72.314713,38.380967,37.618643,38.754115,...,54.166539,65.598887,72.703722,37.245496,74.195478,50.303165,70.091687,51.561183,78.268383,193 192 193 194 194 194 193 192 168 111 50 12 ...
4,66.725301,39.621261,32.244810,38.042032,58.565890,39.621261,72.515926,39.884466,36.982380,39.094852,...,64.889521,60.671411,77.523239,31.191755,76.997301,44.962748,73.707387,44.227141,86.871166,147 148 160 196 215 214 216 217 219 220 206 18...


In [4]:
test.head()

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [5]:
label.head()

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN


##### 空值处理

In [6]:
train.isnull().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [7]:
train.ffill(inplace=True)
train.isnull().sum()

left_eye_center_x            0
left_eye_center_y            0
right_eye_center_x           0
right_eye_center_y           0
left_eye_inner_corner_x      0
left_eye_inner_corner_y      0
left_eye_outer_corner_x      0
left_eye_outer_corner_y      0
right_eye_inner_corner_x     0
right_eye_inner_corner_y     0
right_eye_outer_corner_x     0
right_eye_outer_corner_y     0
left_eyebrow_inner_end_x     0
left_eyebrow_inner_end_y     0
left_eyebrow_outer_end_x     0
left_eyebrow_outer_end_y     0
right_eyebrow_inner_end_x    0
right_eyebrow_inner_end_y    0
right_eyebrow_outer_end_x    0
right_eyebrow_outer_end_y    0
nose_tip_x                   0
nose_tip_y                   0
mouth_left_corner_x          0
mouth_left_corner_y          0
mouth_right_corner_x         0
mouth_right_corner_y         0
mouth_center_top_lip_x       0
mouth_center_top_lip_y       0
mouth_center_bottom_lip_x    0
mouth_center_bottom_lip_y    0
Image                        0
dtype: int64

In [8]:
m, n = train.shape
print("{},{}".format(m, n))

7049,31


In [9]:
test.shape

(1783, 2)

In [10]:
print(math.sqrt(len(np.array(train["Image"][0].split(" "), dtype='float64'))))

96.0


In [11]:
img_len = 96  # 经计算得图片大小为96×96
img = []
for i in range(m):
    splitting = np.array(train["Image"][i].split(" "), dtype='float64')
    splitting = splitting.reshape(img_len, img_len, 1)
    splitting = splitting / 255
    img.append(splitting)
img = np.array(img)
print(img.shape)

(7049, 96, 96, 1)


In [12]:
x_train = img
train.drop(['Image'], axis=1, inplace=True)
y_train = []
for i in range(len(train)):
    y = train.iloc[i,:].values
    y_train.append(y)
y_train= np.array(y_train, dtype='float')

In [13]:
model = Model()
summary(model, input_size=(1, img_len, img_len))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 96, 96]           1,280
              ReLU-2          [-1, 128, 96, 96]               0
       BatchNorm2d-3          [-1, 128, 96, 96]             256
         MaxPool2d-4          [-1, 128, 48, 48]               0
            Conv2d-5          [-1, 128, 48, 48]         147,584
              ReLU-6          [-1, 128, 48, 48]               0
           Dropout-7          [-1, 128, 48, 48]               0
       BatchNorm2d-8          [-1, 128, 48, 48]             256
         MaxPool2d-9          [-1, 128, 24, 24]               0
           Conv2d-10           [-1, 64, 24, 24]          73,792
             ReLU-11           [-1, 64, 24, 24]               0
          Dropout-12           [-1, 64, 24, 24]               0
      BatchNorm2d-13           [-1, 64, 24, 24]             128
        MaxPool2d-14           [-1, 64,

In [14]:
optimizer = Adam(model.parameters(), lr=0.001)
loss_fc = nn.MSELoss()

# 将数据转换为 PyTorch 张量
x_train = torch.tensor(x_train, dtype=torch.float32).reshape(-1, 1, 96, 96)
y_train = torch.tensor(y_train, dtype=torch.float32)

# 创建数据加载器
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [15]:
# 训练模型
epochs = 2
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fc(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/2, Loss: 101.39218799452976
Epoch 2/2, Loss: 20.634095282576205


In [16]:
test_images = []
for i in range(len(test)):
    item = np.array(test['Image'][i].split(' '), dtype='float')
    item = np.reshape(item, (img_len, img_len, 1))
    item /= 255
    test_images.append(item)
    
test_images_np = np.stack(test_images)
test_images_np = test_images_np.transpose(0, 3, 1, 2)
test_images_tensor = torch.from_numpy(test_images_np).float()

In [17]:
predict = model(test_images_tensor)
print(predict)

tensor([[65.6173, 37.7306, 30.0147,  ..., 71.8341, 44.2084, 76.4923],
        [57.9341, 34.2906, 28.4257,  ..., 66.4190, 42.1775, 68.7340],
        [67.4705, 38.2994, 30.5562,  ..., 79.0829, 50.2908, 79.9191],
        ...,
        [75.9162, 40.7280, 33.8074,  ..., 86.1830, 53.5663, 87.9104],
        [69.8754, 36.7631, 30.0935,  ..., 80.1919, 50.8002, 83.9034],
        [65.0259, 38.7525, 30.3205,  ..., 71.7482, 46.6410, 75.9955]],
       grad_fn=<AddmmBackward0>)


In [18]:
IdLookupTable = pd.read_csv('IdLookupTable.csv')
feature_names = list(IdLookupTable['FeatureName'])
image_ids = list(IdLookupTable['ImageId']-1)
row_ids = list(IdLookupTable['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))

In [22]:
print(feature_names)

['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'left_eye_inner_corner_x', 'left_eye_inner_corner_y', 'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 'right_eye_inner_corner_x', 'right_eye_inner_corner_y', 'right_eye_outer_corner_x', 'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y', 'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x', 'mouth_right_corner_y', 'mouth_center_top_lip_x', 'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y', 'left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'left_eye_inner_corner_x', 'left_eye_inner_corner_y', 'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 'right_eye_inner_corner_x', 'right_eye_

In [21]:
print(feature_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 

In [32]:
predictions = []
for x,y in zip(image_ids, feature_list):
    prediction_tensor = predict[x][y]
    prediction_numpy = prediction_tensor.detach().numpy()
    predictions.append(prediction_numpy)
print(predictions)

[array(65.61734, dtype=float32), array(37.73064, dtype=float32), array(30.014725, dtype=float32), array(37.916832, dtype=float32), array(57.734356, dtype=float32), array(36.556004, dtype=float32), array(71.219536, dtype=float32), array(35.023567, dtype=float32), array(33.88432, dtype=float32), array(38.41841, dtype=float32), array(19.187263, dtype=float32), array(40.382374, dtype=float32), array(53.315315, dtype=float32), array(30.447908, dtype=float32), array(77.46354, dtype=float32), array(27.765387, dtype=float32), array(39.023224, dtype=float32), array(30.846493, dtype=float32), array(16.402365, dtype=float32), array(33.070263, dtype=float32), array(48.488613, dtype=float32), array(60.14052, dtype=float32), array(70.11532, dtype=float32), array(68.268326, dtype=float32), array(33.266262, dtype=float32), array(75.43549, dtype=float32), array(48.87908, dtype=float32), array(71.83408, dtype=float32), array(44.20836, dtype=float32), array(76.49235, dtype=float32), array(57.934063, dtyp

In [33]:
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name='Location')
locations = locations.clip(0.0, 96.0)

submission_result = pd.concat([row_ids,locations],axis = 1)

In [ ]:
submission_result.to_csv('submission.csv',index = False)